# Clustering Antarctic Penguin Species - Datacamp Project

Arctic Penguin Exploration: Unraveling Clusters in the Icy Domain with K-means Clustering

## Project Description
Delve into the information about penguins by utilizing unsupervised learning techniques on a thoughtfully curated dataset. uncover concealed patterns, clusters, and relationships that exist within the dataset.

---

You have been asked to support a team of researchers who have been collecting data about penguins in Antartica! The data is available in csv-Format as `penguins.csv`

**Origin of this data** : Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network.

**The dataset consists of 5 columns.**

Column | Description
--- | ---
culmen_length_mm | culmen length (mm)
culmen_depth_mm | culmen depth (mm)
flipper_length_mm | flipper length (mm)
body_mass_g | body mass (g)
sex | penguin sex

Unfortunately, they have not been able to record the species of penguin, but they know that there are **at least three** species that are native to the region: **Adelie**, **Chinstrap**, and **Gentoo**.  Your task is to apply your data science skills to help them identify groups in the dataset!

In [1]:
# Import Required Packages
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Loading and examining the dataset
penguins_df = pd.read_csv("penguins.csv")
penguins_df.head()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,39.1,18.7,181.0,3750.0,MALE
1,39.5,17.4,186.0,3800.0,FEMALE
2,40.3,18.0,195.0,3250.0,FEMALE
3,NaN,NaN,NaN,NaN,NaN
4,36.7,19.3,193.0,3450.0,FEMALE


Before going to the creation of dummy variables it is a good practice to understand how much NaN we have on our dataset.

In [3]:
missing_counts = penguins_df.isna().sum()
missing_percentage = (missing_counts/len(penguins_df))*100
missing_summary = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Percentage (%)': missing_percentage})
print(missing_summary)

                   Missing Count  Missing Percentage (%)
culmen_length_mm               2                0.581395
culmen_depth_mm                2                0.581395
flipper_length_mm              2                0.581395
body_mass_g                    2                0.581395
sex                            9                2.616279


Missing values is very low so let's drop the rows contaiing NaN.

In [10]:
penguins_cleaned = penguins_df.dropna()
penguins_cleaned.head()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,39.1,18.7,181.0,3750.0,MALE
1,39.5,17.4,186.0,3800.0,FEMALE
2,40.3,18.0,195.0,3250.0,FEMALE
4,36.7,19.3,193.0,3450.0,FEMALE
5,39.3,20.6,190.0,3650.0,MALE


1. Perfrom preprocessing steps in the dataset to create dummy variables

In [11]:
penguins_with_dummies = pd.get_dummies(penguins_cleaned, columns=['sex'], drop_first=True)
penguins_with_dummies.head()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex_FEMALE,sex_MALE
0,39.1,18.7,181.0,3750.0,False,True
1,39.5,17.4,186.0,3800.0,True,False
2,40.3,18.0,195.0,3250.0,True,False
4,36.7,19.3,193.0,3450.0,True,False
5,39.3,20.6,190.0,3650.0,False,True
